In [1]:
import os
import glob, pylab, pandas as pd
import pydicom, numpy as np
import random
import json
import time
# import copy
# import pydicom
import torchvision
import sys
sys.path.append('../')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from matplotlib import patches, patheffects

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from pathlib import Path
from dataset import RsnaDataset, train_transforms, val_transforms, test_transforms
from loss import FocalLoss, F1_Loss
# from train import one_epoch_train, model_eval
from resnet import *
from main import *

In [2]:
import easydict 
args = easydict.EasyDict({ "batch_size": 8, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'workers':4,
                         'output_dir' :'../trained_models/retinanet/'})

In [3]:
PATH = '../data/'
df = pd.read_csv(PATH + 'stage_2_train_labels.csv')
df.tail()

,patientId,x,y,width,height,Target
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1
30226,c1f7889a-9ea9-4acb-b64c-b737c929599a,233.0,424.0,201.0,356.0,1


In [4]:
print(len(os.listdir(PATH + 'stage_2_train_images')))
df = df.drop_duplicates('patientId').reset_index(drop=True)
print(df.shape)
print(df.head())

26684
(26684, 6)
                              patientId      x      y  width  height  Target
0  0004cfab-14fd-4e49-80ba-63a80b6bddd6    NaN    NaN    NaN     NaN       0
1  00313ee0-9eaa-42f4-b0ab-c148ed3241cd    NaN    NaN    NaN     NaN       0
2  00322d4d-1c29-4943-afc9-b6754be640eb    NaN    NaN    NaN     NaN       0
3  003d8fa0-6bf1-40ed-b54c-ac657f8495c5    NaN    NaN    NaN     NaN       0
4  00436515-870c-4b36-a041-de91049b9ab4  264.0  152.0  213.0   379.0       1


In [5]:
# df = df[0:2000]
train_df, test_df = train_test_split(df, test_size=0.1, random_state=0)

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
print(train_df.shape)
print(test_df.shape)

print(len(df[df['Target']==1])/len(df))
print(len(train_df[train_df['Target']==1])/len(train_df))
print(len(test_df[test_df['Target']==1])/len(test_df))

(24015, 6)
(2669, 6)
0.225303552690751
0.22477618155319593
0.23004870738104158


In [6]:
train_df.head()

,patientId,x,y,width,height,Target
0,d0183e7f-16b5-4dec-9190-2141bc78f683,NaN,NaN,NaN,NaN,0
1,fdeff9e3-54ca-487e-a00b-71c9072eed25,NaN,NaN,NaN,NaN,0
2,6b033062-62d0-4506-8dda-d3f5df1ee117,NaN,NaN,NaN,NaN,0
3,9ec108de-18b4-4446-a6e8-4c398f5a6614,209.0,429.0,187.0,204.0,1
4,f01452c9-636e-47e9-b126-33be9892fdc5,NaN,NaN,NaN,NaN,0


In [7]:
BATCH_SIZE = 2
train_dataset = RsnaDataset(train_df, transforms=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = RsnaDataset(test_df, transforms=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [8]:
pimage, label = next(iter(train_loader))
label

tensor([0, 0])

In [9]:
# backbone_model= torchvision.models.resnet101(pretrained=True)
# model = resnet101(pretrained=False)
# model.conv1.weight[0,0,0]

In [10]:
# model.load_state_dict(backbone_model.state_dict(), strict=False)
# model.conv1.weight[0,0,0]

In [11]:
backbone_model= torchvision.models.resnet101(pretrained=True)
model = resnet101(pretrained=False)
print(model.conv1.weight[0,0,0])
model.load_state_dict(backbone_model.state_dict(), strict=False)
print(model.conv1.weight[0,0,0])

num_ftrs = model.fc1.in_features
print(num_ftrs)
model.fc1 = nn.Linear(num_ftrs, 2) # target label is 2

# # Load ViT
# from pytorch_pretrained_vit import ViT
# model = ViT('B_16_imagenet1k', pretrained=True, num_classes=2)

# criterion = nn.CrossEntropyLoss()
criterion = FocalLoss(alpha=0.97, gamma=2, reduce=True)
# criterion = F1_Loss()
# model_ft = model.cuda()

# # save for ensemble
# default_model = copy.deepcopy(model.state_dict())

# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda')
model.to(device)

criterion = criterion.to(device)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / (epoch + 1)
)

tensor([ 0.0335, -0.0539,  0.0077, -0.0099, -0.0155, -0.0056, -0.0021],
       grad_fn=<SelectBackward>)
tensor([ 0.0202, -0.0044, -0.0183, -0.0284, -0.0152, -0.0016,  0.0093],
       grad_fn=<SelectBackward>)
256


In [12]:
# test_acc, f1_loss = model_eval(model, test_loader, criterion)

In [ ]:
best_acc1 = 0
acc1 = 0
train_loss = []
val_acc = []
for epoch in range(args.start_epoch, args.epochs):
    adjust_learning_rate(optimizer, epoch, args)

    # train for one epoch
    epoch_loss = train(train_loader, model, criterion, optimizer, epoch, args)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, args)  
    
    train_loss.append(epoch_loss)
    val_acc.append(acc1)
    print('************train_loss {} val_acc {}*************'.format(epoch_loss, acc1))
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

#     if not args.multiprocessing_distributed or (args.multiprocessing_distributed
#             and args.rank % ngpus_per_node == 0):
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': args.arch,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best, args, filename='../trained_model/checkpoint.pt')    

/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Epoch: [0][    0/12008]	Time  0.363 ( 0.363)	Data  0.165 ( 0.165)	Loss 3.2124e-01 (3.2124e-01)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)


In [ ]:
from matplotlib import pyplot as plt
train_lo, test_lo = losses
epochs = range(num_epochs)
# list(train_lo)
plt.plot(epochs, train_lo)

plt.plot(epochs, test_lo)
plt.xlabel('Epochs')
plt.ylabel('Loss')


plt.legend(['Train', 'Test'])
plt.show()